In [6]:
!pip install git+https://github.com/sladkovm/strava-swagger-client

  Cloning https://github.com/sladkovm/strava-swagger-client to c:\users\james\appdata\local\temp\pip-req-build-nh_rxhs3
  Resolved https://github.com/sladkovm/strava-swagger-client to commit 9f311d40d2fb715dcd098e5eaa6b714c5e70853b
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for swagger-client: started
  Running setup.py install for swagger-client: finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/sladkovm/strava-swagger-client 'C:\Users\James\AppData\Local\Temp\pip-req-build-nh_rxhs3'
  DEPRECATION: swagger-client is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


API initialization code

In [56]:

#from __future__ import print_statement
import time
import swagger_client
from swagger_client.rest import ApiException
from pprint import pprint

configuration = swagger_client.Configuration()
configuration.access_token = 'd4d834df8cf82ee2e739ea432818f917a70d14f9'

# create an instance of the API class
api_instance = swagger_client.SegmentsApi(swagger_client.ApiClient(configuration))


In [17]:
import math
def get_bounds_in_area(starting_lat,starting_long,distance):
    # returns southwest corner and northeast corner
  top_right =   MaxLatLongOnBearing(starting_lat, starting_long,45,distance)
  bottom_right = MaxLatLongOnBearing(starting_lat, starting_long, 135, distance)
  bottom_left = MaxLatLongOnBearing(starting_lat, starting_long, 225, distance)
  top_left = MaxLatLongOnBearing(starting_lat, starting_long, 315, distance)
  return {"top right":top_right,"bottom right":bottom_right,"bottom left":bottom_left,"top left" :top_left}

def MaxLatLongOnBearing(centerLat, centerLong, bearing, distance):
    

        lon_rads = math.radians(centerLong)
        lat_rads = math.radians(centerLat)
        bearing_rads = math.radians(bearing)
        max_lat_rads = math.asin(math.sin(lat_rads) * math.cos(distance / 6371) + math.cos(lat_rads) * math.sin(distance / 6371) * math.cos(bearing_rads))
        max_lon_rads = lon_rads + math.atan2((math.sin(bearing_rads) * math.sin(distance / 6371) * math.cos(lat_rads)), (math.cos(distance / 6371) - math.sin(lat_rads) * math.sin(max_lat_rads)))

        maxLat = math.degrees(max_lat_rads)
        maxLong = math.degrees(max_lon_rads)

        return (maxLat,maxLong)
    



In [18]:
CO_Springs_lat_and_lon = (38.833881,-104.821365)
get_bounds_in_area(CO_Springs_lat_and_lon[0],CO_Springs_lat_and_lon[1],5)

{'top right': (38.865669713496985, -104.78052884172236),
 'bottom right': (38.80207808250494, -104.78056531050585),
 'bottom left': (38.80207808250494, -104.86216468949414),
 'top left': (38.865669713496985, -104.86220115827764)}

In [52]:
def get_top_ten_segments_in_area(lat,lon,distance):
    area_sqaure = get_bounds_in_area(lat,lon,distance)
    bounds =  [area_sqaure["bottom left"][0],area_sqaure["bottom left"][1],area_sqaure["top right"][0],area_sqaure["top right"][1]]# array[Float] | The latitude and longitude for two points describing a rectangular boundary for the search: [southwest corner latitutde, southwest corner longitude, northeast corner latitude, northeast corner longitude]
    activity_type = "running" # String | Desired activity type. (optional)

    try: 
        # Explore segments
        api_response = api_instance.explore_segments(bounds, activity_type=activity_type)
        return api_response.to_dict()
    except ApiException as e:
        print("Exception when calling SegmentsApi->exploreSegments: %s\n" % e)
        return None

In [54]:
import json
def save_to_json_file(filename,dict):
    # check for file already there
    try:
        segment_data_dict = json.loads(filename)
    except Exception as e:
        segment_data_dict = None
    # append to file if already there
    if segment_data_dict:
        segment_data_dict["segments"].append(dict["segments"])
    else:
        segment_data_dict = dict
    json_object = json.dumps(segment_data_dict)
    # Writing to sample.json
    with open(filename, "w+") as outfile:
        outfile.write(json_object)


Saving a sample api call to json file

In [57]:
api_resp = get_top_ten_segments_in_area(CO_Springs_lat_and_lon[0],CO_Springs_lat_and_lon[1],5)
segment_data_file_name = "segment_data.json"
if api_resp:
    save_to_json_file(segment_data_file_name,api_resp)
   